# Modellierungsprojekt Optimierungsverfahren, Modellierung und Simulation

## Organisatorisches
In dieser Übung sollen Sie verschiedene dynamische Energiespeicherprobleme als gemischt-ganzzahlige Programme modellieren, mit Gurobi lösen und die Lösung visualisieren und interpretieren.

Die Abgabe erfolgt in Form eines Jupyter-Notebooks. Nutzen Sie den Markup-Modus, um ihre mathematische Modellierung zu beschreiben. Mathematische Formeln können Sie zwischen zwei `$`-Zeichen oder zwei doppelten Dollarzeichen `$$` setzen. Sie können auch per Copy-and-paste Markup-Codeschnipsel kopieren, z.B. aus den Übungsblättern.

Die Abgabe erfolgt in Teams von bis zu 4 Personen, die in ILIAS registriert werden müssen. Die Notebooks müssen bis **Montag, 19.06.2022, 18:00** in ILIAS abgegeben werden. 

Die Veranstaltungstermine am 09.06. sowie in der Woche ab dem 12.06. sind für die Arbeit am Projekt vorgesehen. Ich bin während der Termine vor Ort und stehe für Rückfragen und Unterstützung zur Verfügung. In den Veranstaltungsterminen in der Woche ab dem 19.06.2023 werde ich Sie in Einzelgesprächen (in Präsenz) zu ihrer abgegebenen Lösung befragen (ca. 10-15 Minuten). Die Termine bekommen Sie separat mitgeteilt.

Insgesamt gibt es 100 Punkte auf das Projekt, wobei 50 Punkte für eine 4.0 erforderlich sind. Folgende Aspekte werden für die Punktevergabe berücksichtigt:
- Korrektheit des mathematischen Modells
- Korrektheit der Implementierung
- Interpretation der Ergebnisse
- Darstellung, Übersichtlichkeit und Verständlichkeit des abgegebenen Jupyter Notebooks
- Einzelgespräche

Die Punktzahlen der einzelnen Aufgaben (z.B. 10+5) beziehen sich auf Punkte für die Abgabe plus Punkte für die (mündliche) Beantwortung von Fragen zu den Aufgaben in den Einzelgesprächen. Mögliche Fragen sind z.B.:

- "Wie haben Sie Bedingung X aus der Aufgabe im Modell realisiert?"
- "Was bedeutet die Constraint Y in Ihrem Modell?"
- "Was bedeuten die Terme in der Zielfunktion?"
- etc.

Das Projekt macht 30\% der Modulnote aus.

## Ausgangssituation
Sie sind Assistent der Geschäftsführung in einem mittelständischen Unternehmen. An einem Standort Ihrer Firma wurde kürzlich eine Photovoltaikanlage auf den Dächern der Gebäude sowie ein 100 kWh Stromspeicher (Batterie) installiert. Die PV-Anlage soll vorrangig dafür verwendet werden, um den Strombedarf des Standorts zu decken. Wenn die PV-Anlage mehr Strom produziert als der Standort gerade benötigt, wird damit der Stromspeicher geladen oder der Strom ins Netz eingespeist. Für das Einspeisen bekommen Sie eine Vergütung von 4 ct/kWh. Falls der Standort mehr Strom benötigt als die PV-Anlage produziert und die Batterie leer ist, so beziehen Sie Strom für 22 ct/kWh aus dem Netz. Schematisch sieht das Ganze so aus:

<div>
<img src="Aufgabe1.png" width="750"/>
</div>

Die Batterie hat eine Effizienz von 90% beim Be- und Entladen. Das bedeutet, dass für jede kWh Strom, die in der Batterie gespeichert wird, der Ladezustand um 0.9 kWh steigt. Umgekehrt kann mit einer 1 kWh aus der Batterie nur 0.9 kWh Bedarf gedeckt werden. Die Batterie kann mit einer Maximalleistung von 50kW ge- bzw. entladen werden, d.h. während einer Stunde muss die Summe aller Lastflüsse in und aus der Batterie kleiner als 50 kWh sein.

Ihr Chef hat gelesen, dass es mit Hilfe eines dynamischen Strompreistarifs möglich ist, den Betrieb der Anlage und die Energieversorgung des Standorts zu optimieren. Er bittet Sie, der Sache nachzugehen.

## ✏ Aufgabe 0: Datenanalyse (5 Punkte)
Sie möchten zunächst das System abbilden, wie es heute arbeitet. Dafür besorgen Sie sich historische Daten über den Stromverbrauch des Standorts (`Last [kWh]`) sowie Daten über die Sonneneinstrahlung, aus der Sie die Stromproduktion der PV-Anlage errechnen (`Solar [kWh]`). Alle Daten sind stundengenau für ein Jahr in der Datei `energy_2020.pkl` hinterlegt. In der Spalte `Preis [EUR/kWh]` ist zusätzlich der variable Strompreis für jede Stunde gegeben. Diesen benötigen Sie erst später.

In [4]:
import pandas as pd
df = pd.read_pickle("energy_2020.pkl")
df

,Datum (MEZ),Solar [kWh],Preis [EUR/kWh],Last [kWh]
0,2020-01-01T00:00+01:00,0.0,0.20940,94.2636
1,2020-01-01T01:00+01:00,0.0,0.19300,92.4732
2,2020-01-01T02:00+01:00,0.0,0.18275,90.2108
3,2020-01-01T03:00+01:00,0.0,0.16160,87.5294
4,2020-01-01T04:00+01:00,0.0,0.15425,86.8490
...,...,...,...,...
8779,2020-12-31T19:00+01:00,0.0,0.29735,120.8536
8780,2020-12-31T20:00+01:00,0.0,0.28350,112.4558
8781,2020-12-31T21:00+01:00,0.0,0.26220,106.1954
8782,2020-12-31T22:00+01:00,0.0,0.25930,103.6396


Bearbeiten Sie folgende Aufgaben:
1. Visualiseren Sie die drei Zeitreihen.
1. Wie hoch war der Gesamtstromverbrauch des Standorts im Jahr 2020?
1. Wie hoch wären die Gesamtstromkosten des Standorts im Jahr 2020 ohne PV-Anlage gewesen (d.h. mit 22 ct/kWh Strompreis)?
1. Wie hoch wäre die gesamte produzierte Strommenge der PV-Anlage im Jahr 2020 gewesen?
1. Wie hoch war der durchschnittliche Strompreis im Jahr 2020?

In [ ]:
# TO DO

## ✏ Aufabe 1: Basismodell (25+10 Punkte)
Modellieren Sie das System als lineares Programm, so dass die Gesamtstromkosten minimiert werden. Der Strombedarf des Standorts muss zu jedem Zeitpunkt 100% gedeckt werden. Gehen Sie vereinfachend davon aus, dass die Batterie zu Beginn des betrachteten Zeitraums eine Ladestand von 0 hat. 

1. Formulieren Sie ein lineares Programm in mathematischer Notation an, welches das Problem löst (Indexmengen, Problemdaten, Variablen, Zielfunktion, Nebenbedingungen).
2. Lösen Sie das Problem mit Gurobi, lesen Sie folgende Größen aus der Lösung aus, und visualisieren Sie sie:
   - Ladezustand der Batterie zu jedem Zeitpunkt $t$.
   - Menge an ein- bzw. ausgespeichertem Strom zu jedem Zeitpunkt $t$.
   - Strombezug und Stromverkauf an den Netzbetreiber zu jedem Zeitpunkt $t$.
   - Menge an selbst verbrauchtem Strom aus der PV-Anlage zu jedem Zeitpunkt $t$.
3. Geben Sie außerdem für das Jahr 2020 die folgenden Kenngrößen an:
   - Eigenverbrauch gesamt (in kWh)
   - Eigenverbrauch aus der Batterie (in kWh)
   - Gesamtstromkosten des Standorts mit PV-Anlage und Batterie (in EUR)
   - Ersparnis durch Eigenverbrauch gesamt (in EUR)
   - Ersparnis durch Eigenverbrauch aus der Batterie (in EUR)

In [ ]:
# TO DO

## ✏ Aufgabe 2: Variabler Strompreis (10+5 Punkte)
Sie haben nun eine realistische Baseline, gegen die Sie alle weiteren Optionen vergleichen können. Als nächstes möchten Sie das Potential des dynamischen Strompreistarifes berechnen. Sie haben sich dafür bereits die relevanten historischen Strompreisdaten besorgt (Spalte `Preis [EUR/kWh]`). Der Tarif wirkt zunächst nur in eine Richtung, d.h. der Einkaufspreis ist variabel, aber die Einspeisevergütung liegt nach wie vor bei konstant 4 ct/kWh. Es kann sich durchaus lohnen, Energie aus dem Netz zu beziehen und in die Batterie einzuspeichern, wenn der Strompreis gerade niedrig ist. 

Insgesamt ergibt sich folgendes Bild:

<div>
<img src="Aufgabe2.png" width="750"/>
</div>

Erweitern Sie nun ihr Modell aus Aufgabe 1 und entwickeln Sie ein lineares Programm (mathematische Formulierung, Formulierung als Python Code und Lösung mit Gurobi) für das optimale Ein- und Ausspeichern von Strom in den Batteriespeicher bei variablem Strompreis beim Stromkauf. Berechnen Sie dieselben Kenngrößen wie in Aufgabe 1 und vergleichen Sie die Lösungen. Heben Sie exemplarisch 2-3 typische Situationen (einige Stunden oder Tage) hervor, bei denen sich die Lösungen sichtbar unterscheiden.

In [ ]:
# TO DO

## ✏ Aufgabe 3: Dimensionierung der Batterie (10+5 Punkte)
Simulieren Sie das Einsparpotential für unterschiedliche Größen von Batteriespeichern. Variieren Sie dafür die Batteriegröße in 100 kWh-Schritten zwischen 0 und 1000 kWh und berechnen Sie für jede Größe das Einsparpotential (für jede Batteriegröße ist ein LP zu lösen).

In [ ]:
# TO DO

## ✏ Aufgabe 4: Einspeisung ins Netz (10+5 Punkte)
Sie haben nun ein gutes Verständnis des Einsparpotentials ihrer aktuellen Anlage gewonnen sowie der Potentiale, die sich durch unterschiedliche Batteriespeicher realisieren lassen könnten. Bisher gehen Ihre Modelle von einer konstanten Einspeisevergütung aus. Untersuchen Sie nun auch noch das Potential, das sich bietet, wenn Sie zusätzlich als *Stromlieferant* am Strommarkt teilnehmen:

<div>
<img src="Aufgabe4.png" width="750"/>
</div>

Wir nehmen an, dass dies unter folgenden Bedingungen möglich ist:
- Pro Zeiteinheit müssen Sie entscheiden, ob Sie Strom zum pauschalen Preis einspeisen möchten oder zum variablen Preis.
- Die Einspeisung kann aus der PV-Anlage und/oder der Batterie erfolgen.
- Falls Sie während einer Zeiteinheit zum variablen Preis ins Netz einspeisen möchten, so ist dies nur möglich, wenn Sie mindestens 100 kWh einspeisen (kombiniert aus PV-Anlage und Batterie).
- Falls Sie zum variablen Preis ins Netz einspeisen möchten, so ist dies zum Marktpreis abzüglich 5 ct/kWh möglich.

Erweitern Sie nun ihr Modell aus Aufgabe 2 und entwickeln Sie ein lineares Programm für das optimale Ein- und Ausspeichern von Strom in den Batteriespeicher bei variablem Strompreis beim Stromkauf und -verkauf (mathematische Formulierung, Formulierung als Python Code und Lösung mit Gurobi). Berechnen Sie dieselben Kenngrößen wie in Aufgabe 2 und vergleichen Sie die Lösungen. Heben Sie exemplarisch 2-3 typische Situationen (einige Stunden oder Tage) hervor, bei denen sich die Lösungen sichtbar unterscheiden.

In [ ]:
# TO DO

## ✏ Aufgabe 5: Validierung (10+5 Punkte)
Bei all Ihren Betrachtungen haben Sie bisher einen Aspekt außer Acht gelassen: Die Berechnungen wurden *a posteriori* durchgeführt, d.h. sie wurden im Nachhinein auf bekannten Daten angestellt. In der Realität weiß man aber zum Zeitpunkt der Entscheidung (ein-/ausspeichern) noch nicht, wie sich der Strompreis, der Verbrauch und die produzierte Strommenge zukünftig entwickeln werden. Das so berechnete Einsparpotential ist deshalb wahrscheinlich ein wenig zu optimistisch.

In der Datei `energy_2020_scenarios.pkl` finden Sie 100 verschiedene Szenarien, wie der Strompreis tatsächlich sein könnte. Evaluieren Sie anhand der verschiedenen Szenarien, wie hoch die Stromkosten mit dem optimalen Ein- und Ausspeichern wie in den Aufgaben 1, 2 und 4 berechnet tatsächlich sein könnten:
- Berechnen Sie für jedes Szenario und für jedes der drei Modelle die Stromkosten.
- Visualisieren Sie die Ergebnisse der Simulation in einem geeigneten (vergleichenden) statistischen Plot.

Was müssten Sie zusätzlich beachten, wenn Sie eine derartige Simulation für verschiedene Szenarien des Strombedarfs und der Stromerzeugung machen würden?

In [ ]:
# TO DO